In [1]:
import pandas as pd
import requests

In [4]:
raidDir   = '/Users/amcguire/Documents/PERSONAL/KOL/raidLogs/'
outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/'
runFiles  = ['r1','r2','r3']

In [13]:
# Populate our run dictionary
allRuns = ""

for run, txtLog in enumerate(runFiles):
    # Python is zero-indexed, so we increment run by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    allRuns = allRuns+open('{}{}.txt'.format(raidDir, txtLog)).read()+"\n"

In [24]:
def collectRaiders(runs):
    """ Function to collect all dungeon raiders. Only grabs folks who
        have defeated at least one monster across all ASS runs. """
    
    raiders = []
    
    for row in runs.split('\n'):
        if ('defeated' in row) & (' (#' in row):
            name = row[:row.find(' (#')]
            raiders = list(set(raiders + [name]))
    
    return raiders
            
collectRaiders(allRuns)

['threebullethamburgler',
 'Stary',
 'Name Guy Man',
 'Just Eyes',
 'kirByllAmA',
 'Liora',
 'John Keel',
 'Captain Scotch',
 'themadowl',
 'AeNimUS',
 'violetinsane',
 'madowl',
 'HeroBill',
 'tdog21',
 'Phillammon']

In [55]:
def tabulateKills(name, runs):
    """ Function to tabulate how many kills a specific individual had.
        Also outputs defeats, if we actually want that. """
    
    kills  = 0
    killed = 0
    
    for row in runs.split('\n'):
        if (name in row) & ('defeated' in row):
            if 'was defeated' in row:
                tSpent = int(row.split(' (')[2].split(' ')[0])
                killed += tSpent
            else:
                tSpent = int(row.split(' (')[2].split(' ')[0])
                kills  += tSpent
                
    return({'kills':kills,'defeats':killed})

In [48]:
def tabulateLoots(name, runs):
    ''' Function to tabulate loot per individuals participating. Also 
        grabs skills, just so that we have em.'''
    
    loot   = []
    skills = 0
    
    for row in runs.split('\n'):
        if ('distributed' in row) and (name in row):
            distTo = row.split(' to ')[1].split(' (#')[0]
            if distTo == name:
                thisLoot = row.split(' distributed ')[1].split(' to ')[0]
                loot = loot + [thisLoot]
        elif (' used The Machine' in row) and (name in row):
            distTo = row.split(' used The ')[0].split(' (#')[0]
            if distTo == name: skills+=1
        
    # Parse out things we don't want to be considered as "true" loot
    notRealLoot = ['ghost pepper','electric Kool-Aid','skull capacitor',
                   'wriggling severed nose','Hunger™ Sauce','bottle of Bloodweiser']
    finalLoot = [x for x in loot if x not in notRealLoot]
    
    return({'loot':finalLoot,'skills':skills})
            
tabulateLoots('Captain Scotch', allRuns)

{'loot': ["Drunkula's wineglass", 'zombie accordion'], 'skills': 2}

In [64]:
finalDisplay = {}

for name in collectRaiders(allRuns):
    
    finalDisplay[name] = {}
    finalDisplay[name].update(tabulateKills(name, allRuns))
    finalDisplay[name].update(tabulateLoots(name, allRuns))
    finalDisplay[name]['lootCount'] = len(finalDisplay[name]['loot'])
    
finalFrame = pd.DataFrame.from_dict(finalDisplay,orient='index').sort_values(by='kills',ascending=False)
finalFrame = finalFrame.loc[:,['kills','defeats','skills','lootCount','loot']]

finalFrame

,kills,defeats,skills,lootCount,loot
madowl,2101,63,2,5,"[Drunkula's cape, Mayor Ghost's toupee, Covers..."
Captain Scotch,1653,6,2,2,"[Drunkula's wineglass, zombie accordion]"
John Keel,1026,27,1,1,[Unkillable Skeleton's breastplate]
Phillammon,999,19,2,2,"[Drunkula's bell, Gets-You-Drunk]"
threebullethamburgler,577,13,0,2,"[Drunkula's silky pants, Mayor Ghost's cloak]"
Name Guy Man,575,1,1,1,[Quiets-Your-Steps]
Liora,548,7,0,1,[Great Wolf's left paw]
violetinsane,548,0,0,0,[]
kirByllAmA,452,1,1,0,[]
Stary,378,4,0,0,[]


In [66]:
from datetime import date

currDate = str(date.today().strftime('%m%d'))
finalFrame.to_csv('{}dungeonLogs{}.csv'.format(outputDir,currDate))